<h1>CI HW4<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1">Introduction</a></span></li><li><span><a href="#Data-Preprocessing" data-toc-modified-id="Data-Preprocessing-2">Data Preprocessing</a></span></li><li><span><a href="#GA-for-Variable-Selection" data-toc-modified-id="GA-for-Variable-Selection-3">GA for Variable Selection</a></span></li><li><span><a href="#Experiment-Result" data-toc-modified-id="Experiment-Result-4">Experiment Result</a></span></li></ul></div>

## Introduction

In the final exam we were asked to select models within the **"recency effect" family** for prediction on the training set (2008-2011) as well as on the test set (2012). 

The GA has been chosen for variable selection. Cross validation has been implemented for fitness evaluation of each individual and the selected lagged hourly temperature and daily moving average temperature will lead to the lowest MAPE across the validation years. 

The Tao's Vanilla Benchmark Model is as follows:

$$Load=\beta_{0}+\beta_{1}Trend+\beta_{2}Weekday*Hour+\beta_{3}Month+\beta_{4}Month*T+\beta_{5}Month*T^2+\beta_{6}Month*T^3+\beta_{7}Hour*T+\beta_{8}Hour*T^2+\beta_{9}Hour*T^3$$

When the recency effect will be introduced, extra terms will be added to this benchmark model. Here the lagged hourly temperature and the daily moving average temperature have been denoted as $T_h$ and $T_d$ respectively. The following extra term will be added to the model when a lagged hourly temperature ($T_h$) will be introduced:

$$
\beta_{10}Month*T_{h}+\beta_{11}Month*T_{h}^2+\beta_{12}Month*T_{h}^3+\beta_{13}Hour*T_{h}+\beta_{14}Hour*T_{h}^2+\beta_{15}Hour*T_{h}^3
$$

The same way works for the daily moving average temperature ($T_d$). The additional terms incurr `105` more parameters to be estimated when a $T_h$ or $T_d$ is introduced, on top of the benchmark model.  

## Data Preprocessing

The holiday effect, weekend effect and 24 solar terms has been implemented to the data set. 

## GA for Variable Selection

In this exam problem, GA has been chosen to conduct the variable selection. The variable selection has been transformed into a binary problem, where 1 indicates a certain lagged hourly temperature or a certain daily moving average temperature has been chosen to be included into Tao's vanilla benchmark model, and 0 suggests the opposite. 

A nice thing of implementing GA in this case study, is that we can leverage the multiprocessing module in Python to parallelize the training process of each individual model within the population. This greatly shortens the training time and in the meantime allows us to have a larger population size and go through more generations through the searching process.

The detail implementation steps are as follows:
1. **\[Initialization step\]** Generate random population of `16` individuals. I set the maximum lag for hourly temperature to be `72` and the maximum day of moving average temperature to be `7`. The binary encoding has been used and each individual is represented by 72+7=`79` binary genes. Each positive gen means that the corresponding feature is included in the model. As we can tell, there are $2^{79}$ combinations of parameters in total.

2. **\[Evaluate Fitness\]** Evaluate the fitness (MAPE cross the validation years) of each individual in the population.
3. **\[New Population\]** Create a new population by repeating following steps until the new population is complete
    * \[Selection\] Select two parent individuals from a population according to their fitness (the better fitness, the bigger chance to be selected) using Tournament selection with the Tournament size=16 * 20%=`3`.
    * Here the elitism selection is also implemented with elite value=1, which means the best solution from the current population will be carried over to the next generation unchanged, with the rest of the new population filled in by tournament selection, crossover, and mutation. Setting the elite parameter to one will therefore guarantee that the best solution is never lost through selection and propagation, which often improves the convergence of the algorithm. 
    * Since the computation cost for each parameter combination is huge, an elite preservation strategy is also implemented to the normal genetic algorithm, to achieve faster convergence. The steps of the elite preservation strategy is:
        * When choosing the parents, a size of the parents that’s larger than the population is selected, using tournament selection.
        * After the parents went through crossover and mutation operators, select the best fit individuals to form the offspring that’s equal to the population size.
    * \[Crossover\] The crossover probability has been set to `99%`, in order to encourage broadening the search scope. This means 99% of offspring is made by crossover. The remaining, 1-99%=1% of offspring will be made from exact copies of individuals from old population. Also note that the elitism selection won’t be affected by this crossover process. 
    * A two point crossover has been implemented
    * \[Mutation\] The mutation rate is usually chosen to be 1/m, where m is the number of features. With that value for the mutation rate, we mutate one feature of each individual (statistically).In our case, mutation probability has been set as `50%` and one of the genes in the selected individual will be reverted. We won’t worry about losing the best solution. After the crossover, this mutation will apply to the newborn. 
    * \[Accepting\] Place new offspring in a new population. 
4. **\[Replace\]** Use new generated population for a further run of algorithm.
5. **\[Test\]** The end condition has been set as below. If one of the end condition is met, the algorithm will stop and return the best solution in current population.
    * There are equal or over `14` duplicates in the offspring. In which case, the population does not have enough potential to evolve and we can conclude the algorithm reaches convergence. 
    * Or, running after `16` generations

## Experiment Result

The code implementation can be found in the submission folder. 

I ran in total of `16` iterations. There are `35` $T_h$ terms and only `1` $T_d$ term being selected. The minimum average MAPE across the validation period is `2.951%`. The in-sample fit MAPE using 2008-2011 years of data to train is `1.913%`. The average of the above two MAPEs gives `2.432%`.

The selected variables are listed as below:

The selected d values are: ['Td_7']

The selected h values are: ['Th_1', 'Th_2', 'Th_4', 'Th_6', 'Th_8', 'Th_9', 'Th_11', 'Th_16', 'Th_18', 'Th_19', 'Th_22', 'Th_23', 'Th_24', 'Th_27', 'Th_28', 'Th_30', 'Th_43', 'Th_46', 'Th_47', 'Th_49', 'Th_51', 'Th_53', 'Th_54', 'Th_55', 'Th_56', 'Th_58', 'Th_60', 'Th_61', 'Th_62', 'Th_63', 'Th_65', 'Th_66', 'Th_69', 'Th_71', 'Th_72']

**In the final submission worksheet, the first 7 days (167 hours) forecast are not available because of the recency effect. The true value has been used to fill out the predicted values column. **